In [2]:
import json
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Configuration ---
FLASHCARDS_FILE = 'flashcards.json'

# --- Helper Functions ---

def load_flashcards(filepath=FLASHCARDS_FILE):
    """Load flashcards from a JSON file."""
    with open(filepath, 'r') as f:
        return json.load(f)

def save_flashcards(data, filepath=FLASHCARDS_FILE):
    """Save flashcards (with metadata) back to the JSON file."""
    with open(filepath, 'w') as f:
        json.dump(data, f, indent=4)

def initialize_flashcards():
    """
    Initialize SM-2 metadata for each flashcard.
    This function appends the following keys if they do not exist:
      - repetition: number of successful reviews (starting at 0)
      - interval: current interval in days (starting at 0)
      - ef: easiness factor (starting at 2.5)
      - last_review_date: date of the last review (empty initially)
      - next_review_date: set to today’s date (so that all cards are due)
    """
    data = load_flashcards()
    today_str = datetime.today().strftime('%Y-%m-%d')
    for key, card in data.items():
        if 'repetition' not in card:
            card['repetition'] = 0
        if 'interval' not in card:
            card['interval'] = 0
        if 'ef' not in card:
            card['ef'] = 2.5
        if 'last_review_date' not in card:
            card['last_review_date'] = ""
        if 'next_review_date' not in card:
            card['next_review_date'] = today_str
    save_flashcards(data)
    print("Flashcards initialized with SM-2 metadata.")
    return data

def update_flashcard(card, quality, review_date):
    """Compressed SM-2 for 7-day learning windows."""
    review_date_dt = review_date

    if quality < 3:
        card['repetition'] = 0
        card['interval'] = 1
    else:
        card['repetition'] += 1
        if card['repetition'] == 1:
            card['interval'] = 1
        elif card['repetition'] == 2:
            card['interval'] = 2
        else:
            card['interval'] = int(card['interval'] * card['ef'])
        
        # Update EF with a more conservative shift
        card['ef'] += 0.05 * (quality - 3)
        if card['ef'] < 1.3:
            card['ef'] = 1.3
        elif card['ef'] > 2.5:
            card['ef'] = 2.5  # upper bound

    card['last_review_date'] = review_date_dt.strftime('%Y-%m-%d')
    next_review_date = review_date_dt + timedelta(days=card['interval'])
    if (next_review_date - review_date_dt).days > 7:
        # Cap reviews to within the 7-day window
        next_review_date = review_date_dt + timedelta(days=7 - (review_date_dt - datetime.strptime(card['last_review_date'], '%Y-%m-%d')).days)

    card['next_review_date'] = next_review_date.strftime('%Y-%m-%d')
    return card

# --- Global Variables for the Review Session ---
flashcards = {}  # Will hold all flashcards with metadata
due_keys = []    # List of keys for cards due for review
current_index = 0

def prepare_due_flashcards(limit=30):
    """
    Load flashcards and create a list of flashcards that are due for review.
    Then start the review session.
    
    Args:
        limit (int): Maximum number of flashcards to review in this session
    """
    global flashcards, due_keys, current_index
    flashcards = load_flashcards()
    today = datetime.today()
    due_keys = []
    for key, card in flashcards.items():
        next_review_date = datetime.strptime(card['next_review_date'], '%Y-%m-%d')
        if next_review_date <= today:
            due_keys.append(key)
    
    # Limit the number of cards to review
    if len(due_keys) > limit:
        print(f"You have {len(due_keys)} cards due, but will only review {limit} in this session.")
        due_keys = due_keys[:limit]
    
    current_index = 0
    if not due_keys:
        print("No flashcards are due for review today.")
    else:
        print(f"{len(due_keys)} flashcards due for review.")
        show_flashcard()

def show_flashcard():
    """
    Display the current flashcard along with interactive widgets:
    - A button to show the explanation.
    - An input box for entering the quality rating.
    - A submit button to update the card and proceed to the next one.
    """
    global flashcards, due_keys, current_index
    clear_output(wait=True)
    if current_index < len(due_keys):
        key = due_keys[current_index]
        card = flashcards[key]
        
        # Add some top padding and use HTML for better formatting
        display(widgets.HTML(value="<div style='margin-top: 20px;'></div>"))
        
        # Card header and concept with text wrapping
        header = widgets.HTML(
            value=f"<div style='padding: 10px; background-color: #f0f8ff; border-radius: 5px; margin-bottom: 10px;'>"
                  f"<h3 style='margin: 0;'>Flashcard {current_index+1}/{len(due_keys)}: {key}</h3>"
                  f"</div>"
        )
        
        concept = widgets.HTML(
            value=f"<div style='padding: 15px; border-left: 5px solid #4285f4; background-color: #f5f5f5; margin-bottom: 15px;'>"
                  f"<h4 style='margin-top: 0;'>Concept:</h4>"
                  f"<p style='word-wrap: break-word; white-space: normal;'>{card['concept']}</p>"
                  f"</div>"
        )
        
        # Button to reveal the explanation
        answer_button = widgets.Button(
            description="Show Explanation",
            button_style='info',
            layout=widgets.Layout(width='200px')
        )
        
        # Use HTML widget for showing explanation with proper formatting
        explanation_area = widgets.HTML(value="")
        
        def on_answer_button_clicked(b):
            explanation_area.value = (
                f"<div style='padding: 15px; border-left: 5px solid #34a853; background-color: #f5f5f5; margin-top: 10px;'>"
                f"<h4 style='margin-top: 0;'>Explanation:</h4>"
                f"<p style='word-wrap: break-word; white-space: normal;'>{card['explanation']}</p>"
                f"</div>"
            )
                
        answer_button.on_click(on_answer_button_clicked)
        
        # Rating section header
        rating_header = widgets.HTML(
            value="<div style='margin-top: 20px; margin-bottom: 5px;'><h4>Rate your recall (0-5):</h4></div>"
        )
        
        # Input box for quality rating instead of slider
        quality_input = widgets.BoundedIntText(
            value=4, 
            min=0, 
            max=5, 
            description="Quality:", 
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='200px')
        )
        
        # Rating explanation
        rating_guide = widgets.HTML(
            value="<div style='font-size: 0.9em; color: #666; margin-top: 5px;'>"
                 "<p>0 = Complete blackout, 5 = Perfect recall</p>"
                 "</div>"
        )
        
        # Button to submit the quality rating and update the flashcard
        submit_button = widgets.Button(
            description="Submit & Next",
            button_style='success',
            layout=widgets.Layout(width='150px', margin='10px 0')
        )
        
        # Area for feedback after submission
        feedback_area = widgets.HTML(value="")
        
        def on_submit(b):
            quality = quality_input.value
            review_date = datetime.today()
            print(f"Updating card {key} with quality {quality}...")
            update_flashcard(card, quality, review_date)
            print(f"Updated card {key} with quality {quality}.")
            flashcards[key] = card
            
            # Show feedback using HTML formatting
            feedback_area.value = (
                f"<div style='padding: 10px; margin-top: 15px; border-left: 5px solid #fbbc05; background-color: #f5f5f5;'>"
                f"<h4 style='margin-top: 0;'>Card Updated:</h4>"
                f"<p>Next review: {card['next_review_date']}</p>"
                f"<p>Interval: {card['interval']} days</p>"
                f"<p>Easiness: {round(card['ef'], 2)}</p>"
                f"</div>"
            )
            
            # Save progress after each card
            save_flashcards(flashcards)
            
            # Wait a moment before moving to next card
            import time
            time.sleep(1.5)
            next_card()
            
        submit_button.on_click(on_submit)
        
        # Display all elements with proper spacing
        display(header, concept, answer_button, explanation_area, 
                rating_header, quality_input, rating_guide, submit_button, feedback_area)
    else:
        # Review session complete
        display(widgets.HTML(
            value="<div style='padding: 20px; margin-top: 20px; text-align: center; background-color: #d6f5d6; border-radius: 10px;'>"
                 "<h2>🎉 Review session complete! 🎉</h2>"
                 "<p>All cards have been saved with their new review dates.</p>"
                 "</div>"
        ))
        save_flashcards(flashcards)

def next_card():
    """Advance to the next flashcard in the due list and display it."""
    global current_index
    current_index += 1
    show_flashcard()

def reset_flashcards(filepath=FLASHCARDS_FILE):
    """
    Reset all SM-2 metadata in the flashcards.json file.
    This resets repetition, interval, ef, last_review_date, and sets next_review_date to today.
    Only preserves the concept and explanation for each card.
    
    Args:
        filepath (str): Path to the flashcards JSON file
        
    Returns:
        dict: The reset flashcards data
    """
    try:
        # Load the existing flashcards
        with open(filepath, 'r') as f:
            data = json.load(f)
        
        today_str = datetime.today().strftime('%Y-%m-%d')
        reset_count = 0
        
        # Reset metadata for each card but keep concept and explanation
        for key, card in data.items():
            # Store concept and explanation
            concept = card.get('concept', '')
            explanation = card.get('explanation', '')
            
            # Reset the card with only concept and explanation
            data[key] = {
                'concept': concept,
                'explanation': explanation,
                'repetition': 0,
                'interval': 0,
                'ef': 2.5,
                'last_review_date': '',
                'next_review_date': today_str
            }
            reset_count += 1
        
        # Save the reset flashcards back to the file
        with open(filepath, 'w') as f:
            json.dump(data, f, indent=4)
        
        print(f"✅ Successfully reset {reset_count} flashcards!")
        print(f"All cards are now due for review today ({today_str}).")
        return data
    
    except Exception as e:
        print(f"❌ Error resetting flashcards: {e}")
        return None


In [3]:
# Run this cell only for your first run
data = initialize_flashcards()

Flashcards initialized with SM-2 metadata.


In [4]:
# Run this cell only if you want to reset all flashcards
data = reset_flashcards()

✅ Successfully reset 30 flashcards!
All cards are now due for review today (2025-04-07).


In [ ]:
prepare_due_flashcards(limit=3)

HTML(value="<div style='padding: 20px; margin-top: 20px; text-align: center; background-color: #d6f5d6; border…